## Cleaning DSNIS Data:
#### We begin by loading our excel file and reading it into a pandas dataframe

In [1]:
import pandas as pd
import xlsxwriter
file_path = r"C:\Users\Zane Peycke\proj\grid3\data\donnes_cartographie_kitenda_1_copy.xlsx"
df = pd.read_excel(file_path, header=None)

#### Remove all rows that are completely empty
##### (The original excel files contain empty rows between different areas etc. in a single sheet)

In [2]:
df.dropna(how='all', inplace=True)
# We reset the numerical index to find our split points later
df.reset_index(inplace=True)

#### We need to reformat information including nom, arie, zone, date for each group of data. 
##### To do this we will begin by separating blocks when 'Nom du Superviseur' occurs in a sheet
##### We will also temporarily remove the original column header (by filering on N°) and save for later use

In [3]:
index_list = [] # list of indicies that contiain 'Nom du Superviseur'
drop_list = [] # List of indicies that contain our original column header
split_string = 'NOM DU SUPERVISEUR'
drop_string = 'N°'
for index in range(len(df.index)):
    if df.iloc[index].str.contains(drop_string).any() == True:
        drop_list.append(index)
    if df.iloc[index].str.contains(split_string).any() == True:
        index_list.append(index)
# Save our column header, then temporarily drop for initial processing
original_index = df.iloc[drop_list[0]]
original_index = original_index.drop('index')

# We save this as a list because the header may appear more than once
df = df.drop(drop_list)

#### Given the indicies of 'Nom du Superviseur', we create separate dataframes for each section. 
##### Each section contains information on superviseur, aire, zone, date, gps, and all samples

In [4]:
df_list = []
for i in range(len(index_list)-1):
    if i == 0: # For the first case we start at index = 0, and end one row before the next occurance of 'nom...'
        df_current = df.iloc[index_list[i]:(index_list[i+1]-1)]
        df_list.append(df_current)
    elif i == len(index_list)-1: # last block should extend to the end of our sheet
        df_current = df.iloc[index_list[i]-1:]
        df_list.append(df_current)
    else: # In all other cases we start at the occurance of 'nom...' and end on the row before the next occurance
        df_current = df.iloc[(index_list[i]-1):(index_list[i+1]-1)]
        df_list.append(df_current)

#### Associate Nom, Aire, Zone, Data, GPS info with each sample
##### The original info is contained in a header above each set of samples, so we need to reshape these values first.
##### We also define string values to aid in filtering common information


In [5]:
name_string = 'NOM DU SUPERVISEUR' # This is also different in some sheets
aire_string = 'AIRE DE SANTE'
zone_string = 'ZONE DE SANTE'
date_string = 'DATE' # Dates are formatted differently, and will need to be cleaned after
gps_string = 'NUMERO DE GPS' #This is different across sheets
travail_string = 'TRAVAIL DU'
string_list = [name_string,aire_string,zone_string,date_string,gps_string,travail_string]


In [6]:
def value_from_header(series, string_list):
    """ Given a series from the header data, process the series to extract the 
    specific information we will later associate with our samples. 
    Example: if our series is NOM DU SUPERVISEUR, name of person,  TRAVAIL DU TERRAIN ZONE RURALE"
    This function returns name of person. 
    
    The format of this information is not consistent across arie/zones so we filter based on a list of strings.
    """
    series = series.dropna()
    series = series.astype(str)
    series = series.drop('index')
    series.reset_index(inplace=True,drop=True)
    indicies_to_drop = []
    for index in range(len(series)):
        for string in string_list:
            if string in series.iloc[index]:
                indicies_to_drop.append(index)
            
    value = series.drop(indicies_to_drop)
    return value

In [7]:
def add_header_info(df, original_index):
    # header size appears to be consistent across some sheets
    header = df.iloc[0:5]
    df = df.drop(df.index[0:5])
        
    for index in range(len(header.index)):
        if header.iloc[index].str.contains(name_string).any() == True:
            name_series = header.iloc[index]
        elif header.iloc[index].str.contains(aire_string).any() == True:
            aire_series = header.iloc[index]
        elif header.iloc[index].str.contains(zone_string).any() == True:
            zone_series = header.iloc[index]
        elif header.iloc[index].str.contains(date_string).any() == True:
            date_series = header.iloc[index]
        elif header.iloc[index].str.contains(gps_string).any() == True:
            gps_series = header.iloc[index]
   
    # Gather values from header series
    name = value_from_header(name_series, string_list).reset_index(drop=True)
    aire = value_from_header(aire_series, string_list).reset_index(drop=True)
    zone = value_from_header(zone_series, string_list).reset_index(drop=True)
    date = value_from_header(date_series, string_list).reset_index(drop=True)
    gps  = value_from_header(gps_series, string_list).reset_index(drop=True)

    
    # Now rename our columns list
    columns_list = (list(original_index))
    columns_list.insert(0,'index')
    df.columns=columns_list

    # Add values extracted from the header
    df[name_string] = name[0]
    df[aire_string] = aire[0]
    df[zone_string] = zone[0]
    df[date_string] = date[0]
    df[gps_string]  = gps[0]
    df.reset_index(inplace=True,drop=True)
    del df['index']
    return df

### Take our separate dataframes, process then combine

In [8]:
clean_df_list = []
for i in range(len(df_list)):
    df = add_header_info(df_list[i], original_index)
    clean_df_list.append(df)

In [9]:
cleaned_df = pd.concat(clean_df_list)
cleaned_df.reset_index(inplace=True,drop=True)

### Write to excel

In [10]:
import pandas.io.formats.excel
pandas.io.formats.excel.header_style = None

In [11]:
writer = pd.ExcelWriter('test_output.xlsx', engine='xlsxwriter')
workbook  = writer.book
cleaned_df.to_excel(writer, sheet_name='Sheet1')
worksheet = writer.sheets['Sheet1']

format = workbook.add_format()
format.set_align('left')
format.set_align('vcenter')

worksheet.set_column('A:Z',14, format) # 14 is cell width, is there a way to make this auto adjust?
writer.save()

# 